In [130]:
covariance_matrix = [[9, 0, -27, 18],
                     [0, 9, -9, -27],
                     [-27, -9, 99, -27],
                     [18, -27, -27, 121]]

#covariance_matrix = [[11.0, 25.0, -30.0],
#                     [-3.0, 2.0, 50.0],
#                     [-10.0, 100.0, 50.0]]

def convert_matrix_to_decimals(matrix):
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            matrix[i][j] =  Decimal(matrix[i][j])
    return matrix
                     

n = len(covariance_matrix)
import math  #for sqrt function
from decimal import Decimal  # to be more precise in arithmetics instead of floats
import copy

covariance_matrix = convert_matrix_to_decimals(covariance_matrix)

In [131]:
# Task 1
# LU decomposition using Gauss Elimination


def generate_null_matrix(dimension, d2 = 0):
    if d2 == 0 : d2 = dimension
    elementary_matrix = [[] for i in range(dimension)]
    for i in range(dimension):
        for j in range(d2): 
            elementary_matrix[i].append(Decimal(0.0))
    return elementary_matrix


def square_matrix_multiplication(matrix_a, matrix_b):
    result_matrix = generate_null_matrix(len(matrix_a))
    for i in range(n):
        for j in range(n):
            for k in range(n):
                result_matrix[i][j] += matrix_a[i][k] * matrix_b[k][j]
    return result_matrix

def matrix_multiplication(matrix_a, matrix_b):
    result_matrix = generate_null_matrix(len(matrix_a), len(matrix_b[0]))
    for i in range(len(matrix_a)):
        for j in range(len(matrix_b[0])):
            for k in range(len(matrix_a[0])):
                result_matrix[i][j] += matrix_a[i][k] * matrix_b[k][j]
    return result_matrix

def ones_column(diagonal_element, elementary_matrices_array, temporary_matrix):
    el = generate_null_matrix(len(temporary_matrix))
    for i in range(n):
        if(i < diagonal_element):
            el[i][i] = Decimal(1.0) 
        else:
            if(temporary_matrix[i][diagonal_element] == 0.0):
                el[i][i] = Decimal(1.0)
            else:
                el[i][i] = Decimal(1.0) / temporary_matrix[i][diagonal_element]
    elementary_matrices_array.append(el)
    temporary_matrix = square_matrix_multiplication(el, temporary_matrix)
    return elementary_matrices_array, temporary_matrix

def zeroes_column(diagonal_element, elementary_matrices_array, temporary_matrix):
    el = generate_null_matrix(len(temporary_matrix))
    for i in range(n):
        el[i][i] = Decimal(1.0)
        if(i > diagonal_element):
            if(temporary_matrix[i][diagonal_element] == 0):
                el[i][diagonal_element] = Decimal(0.0)
            else:
                el[i][diagonal_element] = Decimal(-1.0)
    elementary_matrices_array.append(el)
    temporary_matrix = square_matrix_multiplication(el, temporary_matrix)
    return elementary_matrices_array, temporary_matrix

def cut_i_row_and_jth_column(matrix, row, column):
    temp = []
    for i in range(len(matrix)):
        if(i != row):
            temp.append(matrix[i].copy())
    for i in range(len(temp)):
        temp[i].pop(column)
    return temp

def determinant(matrix):
    if(len(matrix) == 1):
        return matrix[0][0]
    det = 0
    for i in range(len(matrix)):
        if i % 2 == 0:
            det += matrix[0][i] * determinant(cut_i_row_and_jth_column(matrix, 0, i))
        else:
            det -= matrix[0][i] * determinant(cut_i_row_and_jth_column(matrix, 0, i))
    return det

def add_two_matrices(matrix1, matrix2):
    answer = generate_null_matrix(len(matrix1), len(matrix1[0]))
    for i in range(len(matrix1)):
        for j in range(len(matrix1[0])):
            answer[i][j] = matrix1[i][j] +  matrix2[i][j]
    return answer

def substract_two_matrices(matrix1, matrix2):
    answer = generate_null_matrix(len(matrix1), len(matrix1[0]))
    for i in range(len(matrix1)):
        for j in range(len(matrix1[0])):
            answer[i][j] = matrix1[i][j] -  matrix2[i][j]
    return answer

def transpose(matrix):
    result = generate_null_matrix(len(matrix))
    for i in range(len(matrix)):
        for j in range(i, len(matrix)):
            a = matrix[j][i]
            result[j][i] = matrix[i][j]
            result[i][j] = a
    return result

def inverse(matrix):
    inverse = generate_null_matrix(len(matrix))
    det = determinant(matrix)
    if det == 0:
        print("non-invertible")
        return inverse
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if (i + j) % 2 == 0: 
                inverse[i][j] = determinant(cut_i_row_and_jth_column(matrix, i, j)) / det
            else:
                inverse[i][j] = -determinant(cut_i_row_and_jth_column(matrix, i, j)) / det
    return transpose(inverse)

def round_matrix(matrix):
    temp = []
    for i in range(len(matrix)):
        temp.append(matrix[i].copy())
        
    for i in range(len(temp)):
        for j in range(len(temp[0])):
            temp[i][j] = float(round(temp[i][j], 2))
    return temp

def column(matrix, j):
    result = []
    for i in range(len(matrix)):
        result.append(Decimal(matrix[i][j]))
    return result

def row(matrix, i):
    return matrix[i].copy()

def dot_product(vector1, vector2):
    result = Decimal(0)
    for i in range(len(vector1)):
        result += Decimal(vector1[i]) * Decimal(vector2[i])
    return result

def project_1_on_2(vector1, vector2):
    s = Decimal(dot_product(vector1, vector2)) / Decimal(dot_product(vector2, vector2))
    return [Decimal(i) * s for i in  vector2]

def set_column(matrix, j, vector):
    for i in range(len(matrix)):
        matrix[i][j] = Decimal(vector[i])
    return matrix

def vec1_minus_vec2(vec1, vec2):
    return [Decimal(vec1[i]) - Decimal(vec2[i]) for i in range(len(vec1))]

def scale_vector(vec1, constant):
    return [Decimal(vec1[i]) * Decimal(constant) for i in range(len(vec1))]

def sign(number):
    if number >= 0:
        return Decimal(1.0)
    else:
        return Decimal(-1.0)

In [132]:
elementary_matrices_array = []

Upper_triangular = covariance_matrix

for i in range(n):
    elementary_matrices_array, Upper_triangular = ones_column(i, elementary_matrices_array, Upper_triangular)
    elementary_matrices_array, Upper_triangular = zeroes_column(i, elementary_matrices_array, Upper_triangular)

Lower_triangular = elementary_matrices_array[-1]

for i in range(len(elementary_matrices_array) - 2, -1, -1):
    Lower_triangular = square_matrix_multiplication(Lower_triangular, elementary_matrices_array[i])
Lower_triangular = inverse(Lower_triangular)

In [133]:
print("***Gauss LU decomposition***")
print("\nInitial covariance matrix:")
print(round_matrix(covariance_matrix))
print("\nL matrix:")
print(round_matrix(Lower_triangular))
print("\nU matrix:")
print(round_matrix(Upper_triangular))
print("\nMultiplied:")
print(round_matrix(square_matrix_multiplication(Lower_triangular, Upper_triangular)))


***Gauss LU decomposition***

Initial covariance matrix:
[[9.0, 0.0, -27.0, 18.0], [0.0, 9.0, -9.0, -27.0], [-27.0, -9.0, 99.0, -27.0], [18.0, -27.0, -27.0, 121.0]]

L matrix:
[[9.0, 0.0, 0.0, 0.0], [0.0, 9.0, 0.0, 0.0], [-27.0, -9.0, 9.0, 0.0], [18.0, -27.0, 0.0, 4.0]]

U matrix:
[[1.0, 0.0, -3.0, 2.0], [0.0, 1.0, -1.0, -3.0], [-0.0, 0.0, 1.0, -0.0], [0.0, -0.0, 0.0, 1.0]]

Multiplied:
[[9.0, 0.0, -27.0, 18.0], [0.0, 9.0, -9.0, -27.0], [-27.0, -9.0, 99.0, -27.0], [18.0, -27.0, -27.0, 121.0]]


In [134]:
#Task 2
# A = QR decomposition (or QU), Q - orhogonal, R - Upper triangular
#Firstly Gram-Schmidt:

def Gram_Schmidt_orthogonalization(matrix):
    V = generate_null_matrix(len(matrix))
    for i in range(len(matrix)):
        v_i = column(matrix, i)
        for k in range(i):
            v_i = vec1_minus_vec2(v_i, project_1_on_2(column(matrix, i), column(V, k)))
        set_column(V, i, v_i)
    return V

def Gram_Schmidt_orthonormalization(matrix):
    V = Gram_Schmidt_orthogonalization(matrix)
    for i in range(len(V)):
        set_column(V, i, [Decimal(k) / Decimal(math.sqrt(dot_product(column(V, i), column(V, i)))) for k in column(V, i) ])
    return V

def Gram_Schmidt_R(matrix, Q): 
    R = generate_null_matrix(len(matrix))
    for col in range(len(matrix)):
        for row in range(col + 1):
            R[row][col] = dot_product(column(Q, row), column(matrix, col))
    return R



In [135]:
#QR decomposition
Q = Gram_Schmidt_orthonormalization(covariance_matrix)
R = Gram_Schmidt_R(covariance_matrix, Q)

In [136]:
print("***QR decomposition using Gram-Schmidt***")
print("\nInitial covariance matrix:")
print(round_matrix(covariance_matrix))
print("\nQ matrix of orthonormal basis vectors")
print(round_matrix(Q))
print("\nR matrix - upper triangular matrix")
print(round_matrix(R))
print("\nMultiplied:")
print(round_matrix(square_matrix_multiplication(Q, R)))

***QR decomposition using Gram-Schmidt***

Initial covariance matrix:
[[9.0, 0.0, -27.0, 18.0], [0.0, 9.0, -9.0, -27.0], [-27.0, -9.0, 99.0, -27.0], [18.0, -27.0, -27.0, 121.0]]

Q matrix of orthonormal basis vectors
[[0.27, 0.07, 0.8, -0.53], [0.0, 0.31, 0.51, 0.8], [-0.8, -0.51, 0.31, 0.0], [0.53, -0.8, 0.07, 0.27]]

R matrix - upper triangular matrix
[[33.67, -7.22, -101.02, 91.14], [0.0, 28.96, -33.56, -90.09], [0.0, 0.0, 2.8, 0.27], [0.0, 0.0, 0.0, 1.07]]

Multiplied:
[[9.0, 0.0, -27.0, 18.0], [0.0, 9.0, -9.0, -27.0], [-27.0, -9.0, 99.0, -27.0], [18.0, -27.0, -27.0, 121.0]]


In [137]:
#Task 3
#Let's find eigen values
#QR algorithm with low convergence - 100000 iterations

def eigenvalues_QR_algorithm(matrix):
    A_k = copy.deepcopy(matrix)
    U_k = generate_null_matrix(len(A_k))
    for i in range(len(U_k)):
        U_k[i][i] = Decimal(1.0)

    for k in range(100000):
        Q_k_plus1 = Gram_Schmidt_orthonormalization(A_k)
        R_k_plus1 = Gram_Schmidt_R(A_k, Q_k_plus1)
        A_k = square_matrix_multiplication(R_k_plus1, Q_k_plus1)
        U_k = square_matrix_multiplication(U_k, Q_k_plus1)
    eigenvalues = []
    for i in range(len(A_k)):
        eigenvalues.append(A_k[i][i])
    return eigenvalues, A_k, U_k


In [138]:
eigenvalues, A, Transformation = eigenvalues_QR_algorithm(covariance_matrix)

In [139]:
import numpy as np
print("\nInitial covariance matrix:")
print(round_matrix(covariance_matrix))
print("\nEigenvalues from my algorithm")
print(eigenvalues)
print("\nEigenvalues from numpy")
print(np.linalg.eig(round_matrix(covariance_matrix))[0])
print("\nFinal upper triangular matrix:")
print(round_matrix(A))
print("\nAfter Transformations using QR algorithm, Gram Schmidt U*T*U_transposed which should be equal to initial")
print(round_matrix(square_matrix_multiplication(square_matrix_multiplication(Transformation, A), transpose(Transformation))))


Initial covariance matrix:
[[9.0, 0.0, -27.0, 18.0], [0.0, 9.0, -9.0, -27.0], [-27.0, -9.0, 99.0, -27.0], [18.0, -27.0, -27.0, 121.0]]

Eigenvalues from my algorithm
[Decimal('147.8167697794879365359311610'), Decimal('89.09239869358057208486107169'), Decimal('0.8211967199625457605329929291'), Decimal('0.2696348069692180778425008173')]

Eigenvalues from numpy
[147.81676978  89.09239869   0.82119672   0.26963481]

Final upper triangular matrix:
[[147.82, -0.0, -0.0, -0.0], [-0.0, 89.09, -0.0, -0.0], [-0.0, -0.0, 0.82, 0.0], [0.0, -0.0, 0.0, 0.27]]

After Transformations using QR algorithm, Gram Schmidt U*T*U_transposed which should be equal to initial
[[9.0, 0.0, -27.0, 18.0], [0.0, 9.0, -9.0, -27.0], [-27.0, -9.0, 99.0, -27.0], [18.0, -27.0, -27.0, 121.0]]


In [140]:
#Task 4 
#We can do QR decomposition using Givens transformations

def GivensMatrix(row1, row2, column, matrix):
    G = generate_null_matrix(len(matrix))
    length = Decimal(math.sqrt(math.pow(Decimal(matrix[row1][column]), 2) + math.pow(Decimal(matrix[row2][column]), 2)))
    for i in range(len(G)):
        if(i == row1 or i == row2):
            G[i][i] = Decimal(matrix[row1][column]) / length
        else:
            G[i][i] = Decimal(1.0)
    G[row1][row2] = Decimal(matrix[row2][column]) / length
    G[row2][row1] = Decimal(-matrix[row2][column]) / length
    return G


def QR_decomposition_Givens_method(R):
    Q = generate_null_matrix(len(R))
    for i in range(len(Q)):
        Q[i][i] = Decimal(1.0)  
    for col in range(len(R)):
        for row in range(col + 1, len(R)):
            G = GivensMatrix(col, row, col, R)
            R = square_matrix_multiplication(G, R)
            Q = square_matrix_multiplication(G, Q)
    return transpose(Q), R


In [141]:
Q, R = QR_decomposition_Givens_method(covariance_matrix)

In [142]:
print("\nInitial covariance matrix:")
print(round_matrix(covariance_matrix))
print("\nQ")
print(round_matrix(Q))
print("\nR")
print(round_matrix(R))
print("\nMultiplied:")
print(round_matrix(square_matrix_multiplication(Q,R)))



Initial covariance matrix:
[[9.0, 0.0, -27.0, 18.0], [0.0, 9.0, -9.0, -27.0], [-27.0, -9.0, 99.0, -27.0], [18.0, -27.0, -27.0, 121.0]]

Q
[[0.27, 0.07, 0.8, -0.53], [0.0, 0.31, 0.51, 0.8], [-0.8, -0.51, 0.31, -0.0], [0.53, -0.8, 0.07, 0.27]]

R
[[33.67, -7.22, -101.02, 91.14], [0.0, 28.96, -33.56, -90.09], [0.0, 0.0, 2.8, 0.27], [0.0, 0.0, 0.0, 1.07]]

Multiplied:
[[9.0, -0.0, -27.0, 18.0], [0.0, 9.0, -9.0, -27.0], [-27.0, -9.0, 99.0, -27.0], [18.0, -27.0, -27.0, 121.0]]


In [143]:
#Task 5
#Get upper Hessenberg form

def Upper_Hessenberg(matrix):
    H = copy.deepcopy(matrix)
    PG = generate_null_matrix(len(H))
    for i in range(len(H)):
        PG[i][i] = Decimal(1.0)

    for col in range(len(matrix) - 2):
        for row in range(col + 2, len(matrix)):
            G = GivensMatrix(col + 1, row, col, H)
            H = square_matrix_multiplication(square_matrix_multiplication(G, H),transpose(G))
            PG = square_matrix_multiplication(G, PG)
    return H, PG


In [144]:
H, PG = Upper_Hessenberg(covariance_matrix)

In [145]:
print("\nInitial matrix:")
print(round_matrix(covariance_matrix))
print("\nUpper Hessenberg form")
print(round_matrix(H))
print("\nProduct of Givens transformations:")
print(round_matrix(PG))
print("\nMultiplied:")
print(round_matrix(square_matrix_multiplication(square_matrix_multiplication(transpose(PG), H), PG)))


Initial matrix:
[[9.0, 0.0, -27.0, 18.0], [0.0, 9.0, -9.0, -27.0], [-27.0, -9.0, 99.0, -27.0], [18.0, -27.0, -27.0, 121.0]]

Upper Hessenberg form
[[9.0, 32.45, 0.0, 0.0], [32.45, 130.69, 21.86, 0.0], [0.0, 21.86, 97.56, -4.83], [0.0, -0.0, -4.83, 0.75]]

Product of Givens transformations:
[[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.83, 0.55], [0.0, -0.34, 0.52, 0.78], [0.0, -0.94, -0.19, -0.29]]

Multiplied:
[[9.0, 0.0, -27.0, 18.0], [0.0, 9.0, -9.0, -27.0], [-27.0, -9.0, 99.0, -27.0], [18.0, -27.0, -27.0, 121.0]]


In [146]:
def QR_decomposition_Givens_for_Hessenberg(matrix):
    R = copy.deepcopy(matrix)
    Q = generate_null_matrix(len(R))
    for i in range(len(R)):
        Q[i][i] = Decimal(1.0)
    for i in range(len(R)-1):
        G = GivensMatrix(i, i+1, i, R)
        Q = square_matrix_multiplication(G, Q)
        R = square_matrix_multiplication(G, R)
    return(transpose(Q),R)
        


In [147]:
Q,R = QR_decomposition_Givens_for_Hessenberg(H)

round_matrix(square_matrix_multiplication(Q,R))


[[9.0, 32.45, -0.0, 0.0],
 [32.45, 130.69, 21.86, -0.0],
 [-0.0, 21.86, 97.56, -4.83],
 [0.0, -0.0, -4.83, 0.75]]

In [148]:
#Task 6 
#Hessenberg QR Algorithm O(N^3) - 100 iterations

def HessenbergQR(matrix):
    A = copy.deepcopy(matrix)
    U_k = generate_null_matrix(len(A))
    for i in range(len(U_k)):
        U_k[i][i] = Decimal(1.0)
        
    H_k, PG = Upper_Hessenberg(A)
    
    for i in range(100):
        inverse_pg, R_k = QR_decomposition_Givens_for_Hessenberg(H_k)
        H_k = square_matrix_multiplication(R_k, inverse_pg)
        U_k = square_matrix_multiplication(U_k, inverse_pg)
    eigenvalues = []
    for i in range(len(H_k)):
        eigenvalues.append(H_k[i][i])
    U_k = square_matrix_multiplication(transpose(PG), U_k)
    return(eigenvalues, H_k, U_k)
    




In [149]:
eigenvalues_h, H_k, Transformation_h = HessenbergQR(covariance_matrix)

In [150]:
import numpy as np
print("\nInitial covariance matrix:")
print(round_matrix(covariance_matrix))
print("\nEigenvalues from my algorithm")
print(eigenvalues_h)
print("\nEigenvalues from numpy")
print(np.linalg.eig(round_matrix(covariance_matrix))[0])
print("\nFinal Hessenberg matrix:")
print(round_matrix(H_k))
print("\nAfter Transformations using QR algorithm, U*T*U_transposed which should be equal to initial")
print(round_matrix(square_matrix_multiplication(square_matrix_multiplication(Transformation_h, H_k), transpose(Transformation_h))))


Initial covariance matrix:
[[9.0, 0.0, -27.0, 18.0], [0.0, 9.0, -9.0, -27.0], [-27.0, -9.0, 99.0, -27.0], [18.0, -27.0, -27.0, 121.0]]

Eigenvalues from my algorithm
[Decimal('147.8167697794873860387238088'), Decimal('89.09239869358016994803759675'), Decimal('0.8211967199625452077154838405'), Decimal('0.2696348069692176724956488621')]

Eigenvalues from numpy
[147.81676978  89.09239869   0.82119672   0.26963481]

Final Hessenberg matrix:
[[147.82, 0.0, -0.0, 0.0], [0.0, 89.09, -0.0, -0.0], [0.0, -0.0, 0.82, -0.0], [-0.0, 0.0, -0.0, 0.27]]

After Transformations using QR algorithm, U*T*U_transposed which should be equal to initial
[[9.0, 0.0, -27.0, 18.0], [0.0, 9.0, -9.0, -27.0], [-27.0, -9.0, 99.0, -27.0], [18.0, -27.0, -27.0, 121.0]]


In [151]:
#Task 7 Cholesky decomposition - can work only with symmetric positive definite matrix
def Cholesky_decomposition(matrix):
    if len(matrix) == 1:
        return matrix
    L_matrix = generate_null_matrix(len(matrix))
    L_matrix[0][0] = Decimal(math.sqrt(matrix[0][0]))
    for i in range(1, len(matrix)):
        L_matrix[i][0] = matrix[i][0] / L_matrix[0][0]
        
    for j in range(1, len(matrix)):
        for i in range(j, len(matrix)):
            temp_sum = 0 
            if j == i:
                for p in range(j):
                    temp_sum = temp_sum + L_matrix[j][p] ** 2
                L_matrix[j][j] = Decimal(math.sqrt(matrix[j][j] - temp_sum))
            else:
                for k in range(j):
                    temp_sum = temp_sum + L_matrix[j][k] * L_matrix[i][k]
                L_matrix[i][j] = (matrix[i][j] - temp_sum) / L_matrix[j][j]  
    return L_matrix

In [152]:
L = Cholesky_decomposition(covariance_matrix)
print("\nL matrix:")
print(round_matrix(L))
print("\nL * L^T:")
print(round_matrix(square_matrix_multiplication(L, transpose(L))))
print("\nInitial matrix:")
print(round_matrix(covariance_matrix))



L matrix:
[[3.0, 0.0, 0.0, 0.0], [0.0, 3.0, 0.0, 0.0], [-9.0, -3.0, 3.0, 0.0], [6.0, -9.0, 0.0, 2.0]]

L * L^T:
[[9.0, 0.0, -27.0, 18.0], [0.0, 9.0, -9.0, -27.0], [-27.0, -9.0, 99.0, -27.0], [18.0, -27.0, -27.0, 121.0]]

Initial matrix:
[[9.0, 0.0, -27.0, 18.0], [0.0, 9.0, -9.0, -27.0], [-27.0, -9.0, 99.0, -27.0], [18.0, -27.0, -27.0, 121.0]]


In [153]:
#Task 8 Householder rotation
#Input is list 
def Householder_matrix (v):
    I = generate_null_matrix(len(v))
    for i in range(len(v)):
        I[i][i] = Decimal(1.0)
    return substract_two_matrices(I,  matrix_multiplication( [[Decimal(v[i])] for i in range(len(v))], [scale_vector(v, Decimal(2) / dot_product(v, v))] ))

def QR_decomposition_Householder(matrix):
    R = copy.deepcopy(matrix)
    Householder_matrices = []
    Q = generate_null_matrix(len(matrix))
    for i in range(len(matrix)):
        Q[i][i] = Decimal(1.0)
    for p in range(len(matrix[0]) - 1):
        k = [Decimal(R[i][p]) for i in range(len(R))]
        for i in range(p):
            k[i] = Decimal(0.0)
        length = Decimal(math.sqrt(Decimal(dot_product(k,k))))
        k[p] = k[p] - Decimal(sign(k[p])) * length    
        H = Householder_matrix(k)
        Householder_matrices.append(H)
        R = matrix_multiplication(H, R)
        
    for i in range(len(Householder_matrices)):
        Q = square_matrix_multiplication(Q , Householder_matrices[i])
    return Q, R
        

In [161]:
rect_matrix = [[1, -1, 4],
             [1, 4, -2],
             [1, 4, 2],
             [1, -1, 0]]
Q_H, R_H = QR_decomposition_Householder(rect_matrix)

print("\nInitial covariance matrix:")
print(round_matrix(rect_matrix))
print("\nQ")
print(round_matrix(Q_H))
print("\nR")
print(round_matrix(R_H))
print("\nMultiplied:")
print(round_matrix(matrix_multiplication(Q_H,R_H)))



Initial covariance matrix:
[[1.0, -1.0, 4.0], [1.0, 4.0, -2.0], [1.0, 4.0, 2.0], [1.0, -1.0, 0.0]]

Q
[[0.5, -0.5, 0.5, -0.5], [0.5, 0.5, -0.5, -0.5], [0.5, 0.5, 0.5, 0.5], [0.5, -0.5, -0.5, 0.5]]

R
[[2.0, 3.0, 2.0], [0.0, 5.0, -2.0], [0.0, 0.0, 4.0], [0.0, 0.0, 0.0]]

Multiplied:
[[1.0, -1.0, 4.0], [1.0, 4.0, -2.0], [1.0, 4.0, 2.0], [1.0, -1.0, 0.0]]


In [169]:
def Hessenberg_Householder(matrix):
    Hess = copy.deepcopy(matrix)
    Householder_matrices = []
    PHouse = generate_null_matrix(len(matrix))
    for i in range(len(matrix)):
        PHouse[i][i] = Decimal(1.0)
    for p in range(len(matrix[0]) - 2):
        k = [Decimal(Hess[i][p]) for i in range(len(Hess))]
        for i in range(p + 1):
            k[i] = Decimal(0.0)
        length = Decimal(math.sqrt(Decimal(dot_product(k,k))))
        k[p + 1] = k[p + 1] - Decimal(sign(k[p + 1])) * length    
        H = Householder_matrix(k)
        Householder_matrices.append(H)
        Hess = matrix_multiplication(H, Hess)
        
    for i in range(len(Householder_matrices)):
        PHouse = square_matrix_multiplication(PHouse , Householder_matrices[i])
    return PHouse, Hess

round_matrix(Hessenberg_Householder(covariance_matrix)[1])

[[9.0, 0.0, -27.0, 18.0],
 [32.45, -7.49, -97.35, 89.58],
 [-0.0, -28.9, 33.64, 89.85],
 [-0.0, -0.0, -1.55, -1.04]]

In [156]:
sign(0)

Decimal('1')

In [157]:
scale_vector([1,1,1,1], 4)

[Decimal('4'), Decimal('4'), Decimal('4'), Decimal('4')]